In [1]:
import six
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten
)

from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D
)
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

Using TensorFlow backend.


In [2]:
import os
import csv
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
%matplotlib inline

In [3]:
import imageio

In [4]:
def _bn_relu(input):
    """Helper to build a BN -> relu block
    """
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)


def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f


def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f


def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])


def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """Builds a residual block with repeating bottleneck blocks.
    """
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides,
                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input

    return f


def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                           strides=init_strides,
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                  strides=init_strides)(input)

        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)

    return f


def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf

    Returns:
        A final conv layer of filters * 4
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),
                              strides=init_strides,
                              padding="same",
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1),
                                     strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
        return _shortcut(input, residual)

    return f


def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.

        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved

        Returns:
            The keras `Model`.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.image_dim_ordering() == 'tf':
            input_shape = (input_shape[1], input_shape[2], input_shape[0])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(1, 1))(block)
        flatten1 = Flatten()(pool2)
        dense = Dense(units=num_outputs, kernel_initializer="he_normal",
                      activation="softmax")(flatten1)

        model = Model(inputs=input, outputs=dense)
        return model

    @staticmethod
    def build_resnet_18(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

    @staticmethod
    def build_resnet_34(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_101(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])


In [5]:
with open('../cl3403/pickle/y_train1.pickle', mode = 'rb') as f:
    y_train1 = pickle.load(f)
with open('../cl3403/pickle/y_train2.pickle', mode = 'rb') as f:
    y_train2 = pickle.load(f)
with open('../cl3403/pickle/y_train3.pickle', mode = 'rb') as f:
    y_train3 = pickle.load(f)
with open('../cl3403/pickle/y_train4.pickle', mode = 'rb') as f:
    y_train4 = pickle.load(f)
with open('../cl3403/pickle/y_train5.pickle', mode = 'rb') as f:
    y_train5 = pickle.load(f)
with open('../cl3403/pickle/y_train6.pickle', mode = 'rb') as f:
    y_train6 = pickle.load(f)
with open('../cl3403/pickle/y_train7.pickle', mode = 'rb') as f:
    y_train7 = pickle.load(f)
with open('../cl3403/pickle/y_train8.pickle', mode = 'rb') as f:
    y_train8 = pickle.load(f)
with open('../cl3403/pickle/y_train9.pickle', mode = 'rb') as f:
    y_train9 = pickle.load(f)
with open('../cl3403/pickle/y_train10.pickle', mode = 'rb') as f:
    y_train10 = pickle.load(f)

In [6]:
with open('../cl3403/pickle/X_train1.pickle', mode = 'rb') as f:
    X_train1 = pickle.load(f)
with open('../cl3403/pickle/X_train2.pickle', mode = 'rb') as f:
    X_train2 = pickle.load(f)

In [7]:
with open('../cl3403/pickle/X_train3.pickle', mode = 'rb') as f:
    X_train3 = pickle.load(f)
with open('../cl3403/pickle/X_train4.pickle', mode = 'rb') as f:
    X_train4 = pickle.load(f)
with open('../cl3403/pickle/X_train5.pickle', mode = 'rb') as f:
    X_train5 = pickle.load(f)

In [8]:
X_train = np.vstack((X_train1, X_train2))
X_train = np.vstack((X_train, X_train3))
X_train = np.vstack((X_train, X_train4))
X_train = np.vstack((X_train, X_train5))

In [9]:
y_train = np.hstack((y_train1, y_train2))
y_train = np.hstack((y_train, y_train3))
y_train = np.hstack((y_train, y_train4))
y_train = np.hstack((y_train, y_train5))

In [10]:
train1 = []
train2 = []
for i in range(len(X_train)):
    if (i % 4) != 0:
        train1.append(i)
    else:
        train2.append(i)

In [12]:
X_train_true1 = X_train[train1]

In [13]:
y_train_true1 = y_train[train1]

In [18]:
del X_train, y_train, X_train1, X_train2, X_train3, X_train4, X_train5

In [19]:
with open('../cl3403/pickle/X_train6.pickle', mode = 'rb') as f:
    X_train6 = pickle.load(f)
with open('../cl3403/pickle/X_train7.pickle', mode = 'rb') as f:
    X_train7 = pickle.load(f)

In [20]:
with open('../cl3403/pickle/X_train8.pickle', mode = 'rb') as f:
    X_train8 = pickle.load(f)
with open('../cl3403/pickle/X_train9.pickle', mode = 'rb') as f:
    X_train9 = pickle.load(f)
with open('../cl3403/pickle/X_train10.pickle', mode = 'rb') as f:
    X_train10 = pickle.load(f)

In [21]:
X_train = np.vstack((X_train6, X_train7))
X_train = np.vstack((X_train, X_train8))
X_train = np.vstack((X_train, X_train9))
X_train = np.vstack((X_train, X_train10))

In [22]:
y_train = np.hstack((y_train6, y_train7))
y_train = np.hstack((y_train, y_train8))
y_train = np.hstack((y_train, y_train9))
y_train = np.hstack((y_train, y_train10))

In [23]:
train3 = []
train4 = []
for i in range(len(X_train)):
    if (i % 4) != 0:
        train3.append(i)
    else:
        train4.append(i)

In [25]:
del X_train6, X_train7, X_train8, X_train9, X_train10

In [26]:
X_train_true2 = X_train[train3]

In [31]:
y_train_true2 = y_train[train3]

In [35]:
del X_train

In [36]:
X_train_true = np.vstack((X_train_true1, X_train_true2))
y_train_true = np.hstack((y_train_true1, y_train_true2))

In [39]:
model = ResnetBuilder.build_resnet_34((3, 64, 64), 14951)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
hist = model.fit(X_train_true, y_train_true, batch_size = 2000, epochs = 10) 

Epoch 1/10
698826/698826 [==============================] - 366s 523us/step - loss: 6.0208 - acc: 0.2436
Epoch 2/10
698826/698826 [==============================] - 360s 515us/step - loss: 3.3650 - acc: 0.5019
Epoch 3/10
698826/698826 [==============================] - 358s 512us/step - loss: 2.3915 - acc: 0.6389
Epoch 4/10
698826/698826 [==============================] - 357s 511us/step - loss: 1.8984 - acc: 0.7183
Epoch 5/10
698826/698826 [==============================] - 359s 514us/step - loss: 1.6109 - acc: 0.7697
Epoch 6/10
698826/698826 [==============================] - 358s 512us/step - loss: 1.4207 - acc: 0.8051
Epoch 7/10
698826/698826 [==============================] - 358s 512us/step - loss: 1.2825 - acc: 0.8315
Epoch 8/10
698826/698826 [==============================] - 357s 512us/step - loss: 1.1804 - acc: 0.8510
Epoch 9/10
698826/698826 [==============================] - 360s 515us/step - loss: 1.1035 - acc: 0.8670
Epoch 10/10
698826/698826 [============================

In [41]:
hist = model.fit(X_train_true, y_train_true, batch_size = 2000, epochs = 3) 

Epoch 1/3
698826/698826 [==============================] - 357s 511us/step - loss: 0.9948 - acc: 0.8890
Epoch 2/3
698826/698826 [==============================] - 358s 512us/step - loss: 0.9572 - acc: 0.8964
Epoch 3/3
698826/698826 [==============================] - 358s 512us/step - loss: 0.9239 - acc: 0.9034


In [43]:
hist = model.fit(X_train_true, y_train_true, batch_size = 2000, epochs = 3) 

Epoch 1/3
698826/698826 [==============================] - 357s 511us/step - loss: 0.8911 - acc: 0.9098
Epoch 2/3
698826/698826 [==============================] - 357s 510us/step - loss: 0.8696 - acc: 0.9137
Epoch 3/3
698826/698826 [==============================] - 357s 511us/step - loss: 0.8530 - acc: 0.9165


In [44]:
hist = model.fit(X_train_true, y_train_true, batch_size = 2000, epochs = 2) 

Epoch 1/2
698826/698826 [==============================] - 357s 510us/step - loss: 0.8333 - acc: 0.9201
Epoch 2/2
698826/698826 [==============================] - 358s 513us/step - loss: 0.8172 - acc: 0.9234


In [45]:
y_train_true

array([    0,     0,     0, ..., 14950, 14950, 14950])

In [46]:
del X_train_true, y_train_true

In [47]:
with open('../cl3403/pickle/X_truetest.pickle', mode = 'rb') as f:
    X_truetest = pickle.load(f)
with open('../cl3403/pickle/y_truetest.pickle', mode = 'rb') as f:
    y_truetest = pickle.load(f)

In [48]:
pred = model.predict(X_truetest)

In [49]:
landmark_id_pred = pred.argmax(axis = 1)
confidence = pred.max(axis = 1)
df = pd.DataFrame({'id': np.array(y_truetest),
                   'landmark_id': landmark_id_pred,
                   'confidence': confidence})

In [50]:
df['landmarks'] = df.apply(lambda x: str(x['landmark_id']) + ' ' + str(x['confidence']), axis = 1)

In [52]:
df = df.drop(['confidence', 'landmark_id'], axis = 1)

In [54]:
df_sample = pd.read_csv('sample_submission.csv')[['id']]
df_final = df_sample.merge(df, how = 'left', on = 'id')
df_final.to_csv('submission20.csv', index = False)